In [73]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGConformer
import pandas as pd
from collections import OrderedDict
import math
import importlib
import LSTM_model
importlib.reload(LSTM_model)
from LSTM_model import ShallowLSTMNet
from torchinfo import summary

In [74]:
in_chans = 22
n_classes = 4
input_window_samples = 1125
model = ShallowLSTMNet(in_chans,n_classes,input_window_samples)

In [75]:
print(model)

ShallowLSTMNet(
  (lstm): LSTM(22, 22, num_layers=2, batch_first=True)
  (spatial): Conv2d(1, 20, kernel_size=(22, 1), stride=(1, 1))
  (batch_norm): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 20), stride=(1, 20), padding=0)
  (dropout): Dropout(p=0.7, inplace=False)
  (fc): Linear(in_features=1120, out_features=4, bias=True)
)


In [76]:
summary(model,input_size=(2, 22, 1125),col_names=["input_size", "output_size", "num_params", "kernel_size"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ShallowLSTMNet                           [2, 22, 1125]             [2, 4]                    --                        --
├─LSTM: 1-1                              [2, 1125, 22]             [2, 1125, 22]             8,096                     --
├─Conv2d: 1-2                            [2, 1, 22, 1125]          [2, 20, 1, 1125]          460                       [22, 1]
├─BatchNorm2d: 1-3                       [2, 20, 1, 1125]          [2, 20, 1, 1125]          40                        --
├─AvgPool2d: 1-4                         [2, 20, 1, 1125]          [2, 20, 1, 56]            --                        [1, 20]
├─Dropout: 1-5                           [2, 1120]                 [2, 1120]                 --                        --
├─Linear: 1-6                            [2, 1120]                 [2, 4]                    4,484                     --
Tota